In [84]:
import requests as rq 
from decimal import Decimal
from typing import Dict,List
from dataclasses  import dataclass
import json

## Crawl Data

In [85]:
gas_prices_json = rq.get('https://fcd.terra.dev/v1/txs/gas_prices').json()

In [86]:
swap_rates_uusd_json = rq.get("https://fcd.terra.dev/v1/market/swaprate/uusd").json()

In [87]:
mainnet_token_contract_to_info_dict = rq.get("https://assets.terra.money/cw20/tokens.json").json()['mainnet']

In [88]:
len(mainnet_token_contract_to_info_dict)

37

In [89]:
dashboard_json = rq.get("https://fcd.terra.dev/v1/dashboard").json()
tax_rate = Decimal(dashboard_json['taxRate'])
native_tokens = [item['denom'] for item in dashboard_json['taxCaps']]
token_to_tax_cap : Dict[str,Decimal] = {item['denom']:Decimal(item['taxCap']) for item in dashboard_json['taxCaps']}

In [90]:
all_tokens_list = rq.get("https://api.terraswap.io/tokens").json()
len([item for item in all_tokens_list if item['symbol']=="MIR"])
token_contract_to_decimals = {item['contract_addr']:item['decimals'] for item in all_tokens_list}

In [91]:
len(all_tokens_list)

128

In [92]:
len(mainnet_token_contract_to_info_dict)

37

or query token info with graphql on https://mantle.terra.dev/
```
{
  terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp: WasmContractsContractAddressStore(ContractAddress: "terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp", QueryMsg: "{\"token_info\":{}}"){
    Height
    Result
  }
}
```
or 
```
curl 'https://mantle.terra.dev/' -H 'Accept-Encoding: gzip, deflate, br' -H 'Content-Type: application/json' -H 'Accept: application/json' -H 'Connection: keep-alive' -H 'DNT: 1' -H 'Origin: https://mantle.terra.dev' --data-binary '{"query":"{\n  terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp: WasmContractsContractAddressStore(ContractAddress: \"terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp\", QueryMsg: \"{\\\"token_info\\\":{}}\"){\n    Height\n    Result\n  }\n}"}' --compressed
```

In [93]:
pairs_list_from_terraswap = rq.get("https://api.terraswap.io/pairs").json()['pairs']
len(pairs_list_from_terraswap)

65

In [94]:
pairs_list_from_terraswap[0]

{'asset_infos': [{'token': {'contract_addr': 'terra1q59h4hyxvfpu2hp3v39r8rpl4wykqe7axrc9rr'}},
  {'native_token': {'denom': 'uusd'}}],
 'liquidity_token': 'terra165k229vdtpng40rhdfn3tqtqphwxeyjx5wrwxw',
 'contract_addr': 'terra1xlgl3xvkha2y6mssy9s4qe70sq295825sdmt2q'}

In [95]:
pairs_contract_to_info_from_terra = rq.get("https://assets.terra.money/cw20/pairs.json").json()['mainnet']
len(pairs_contract_to_info_from_terra.keys())

33

In [12]:
agg_pair_list = [item["contract_addr"] for item in pairs_list_from_terraswap]+ list(pairs_contract_to_info_from_terra.keys())
len(list(set(agg_pair_list)))

64

In [13]:
oracle_params_json = rq.get("https://fcd.terra.dev/oracle/parameters").json()
market_params_json = rq.get("https://fcd.terra.dev/market/parameters").json()

In [14]:
market_params_json

{'height': '3819297',
 'result': {'base_pool': '32500000000000.000000000000000000',
  'pool_recovery_period': '49',
  'min_spread': '0.005000000000000000'}}

In [15]:
all_terraswap_contracts = [item['contract_addr'] for item in pairs_list_from_terraswap] + list(pairs_contract_to_info_from_terra.keys())
all_terraswap_contracts = list(set(all_terraswap_contracts))

In [16]:
len(all_terraswap_contracts)

64

In [21]:
# this sends a request per contract pool and its slow
# def query_pool_info(pool_contract_address:str):
#     # https://fcd.terra.dev/wasm/contracts/terra1sndgzq62wp23mv20ndr4sxg6k8xcsudsy87uph/store?query_msg=%7B%22pool%22%3A%7B%7D%7D
#     query_str = '{"pool":{}}'
#     query_url = f"https://fcd.terra.dev/wasm/contracts/{pool_contract_address}/store?query_msg={query_str}"
#     # print("query_url",query_url)
#     return rq.get(query_url).json()
# pair_contract_to_pool_info ={}
# for pool_contract in all_terraswap_contracts:
#     pair_contract_to_pool_info[pool_contract] = query_pool_info(pool_contract)

Better to query all pools at once: 

In [96]:
# curl 'https://mantle.terra.dev/' -H 'Accept-Encoding: gzip, deflate, br' -H 'Content-Type: application/json' -H 'Accept: application/json' -H 'Connection: keep-alive' -H 'DNT: 1' -H 'Origin: https://mantle.terra.dev' --data-binary '{"query":"{\n  terra14hklnm2ssaexjwkcfhyyyzvpmhpwx6x6lpy39s: WasmContractsContractAddressStore(ContractAddress: \"terra14hklnm2ssaexjwkcfhyyyzvpmhpwx6x6lpy39s\", QueryMsg: \"{\\\"pool\\\":{}}\"){\n    Height\n    Result\n  }\n  terra1ldpgn35p9m2ksumh8dksp3edvv4nz9tdj2pwv2: WasmContractsContractAddressStore(ContractAddress: \"terra1ldpgn35p9m2ksumh8dksp3edvv4nz9tdj2pwv2\", QueryMsg: \"{\\\"pool\\\":{}}\"){\n    Height\n    Result\n  }\n}\n"}' --compressed

In [23]:
# https://mantle.terra.dev/
# """{
#   terra14hklnm2ssaexjwkcfhyyyzvpmhpwx6x6lpy39s: WasmContractsContractAddressStore(ContractAddress: "terra14hklnm2ssaexjwkcfhyyyzvpmhpwx6x6lpy39s", QueryMsg: "{\"pool\":{}}"){
#     Height
#     Result
#   }
#   terra1ldpgn35p9m2ksumh8dksp3edvv4nz9tdj2pwv2: WasmContractsContractAddressStore(ContractAddress: "terra1ldpgn35p9m2ksumh8dksp3edvv4nz9tdj2pwv2", QueryMsg: "{\"pool\":{}}"){
#     Height
#     Result
#   }
# }
# """

In [97]:
template = """
  contract_code: WasmContractsContractAddressStore(ContractAddress: "contract_code", QueryMsg: "{\\"pool\\":{}}"){
    Height
    Result
  }"""
query_string = "{"
for c_code in all_terraswap_contracts:
    query_string+= template.replace("contract_code",c_code)+"\n"
query_string+='}'
req_all_pools_info = rq.post('https://mantle.terra.dev/',json={"query":query_string})

In [98]:
req_all_pools_info.json().keys()

dict_keys(['data'])

In [99]:
len(req_all_pools_info.json()['data'].keys())

64

### TerraSwap object models

In [101]:
@dataclass
class TerraswapAssetDetail:
    token_name_or_contract_address:str
    is_native_token:bool
    amount:Decimal
    
    @staticmethod
    def from_json(input_json:dict)->'TerraswapAssetDetail':
        # {'amount': '3078494360163', 'info': {'native_token': {'denom': 'uusd'}}}
        # {'amount': '8189896328','info': {'token': {'contract_addr': 'terra137drsu8gce5thf6jr5mxlfghw36rpljt3zj73v'}}}
        amount = Decimal(input_json.get("amount"))
        is_native = False
        if input_json.get('info').get('native_token') is not None:
            is_native = True
            token_name_or_contract_address = input_json.get('info').get('native_token').get('denom')
        else:
            token_name_or_contract_address = input_json.get('info').get('token').get('contract_addr')
        return TerraswapAssetDetail(token_name_or_contract_address=token_name_or_contract_address,
                                   is_native_token=is_native,
                                   amount=amount)
    
@dataclass
class TerraswapPoolInfo:
    asset_detail_1: TerraswapAssetDetail
    asset_detail_2: TerraswapAssetDetail
    total_share: Decimal
    @staticmethod
    def from_json(input_json:dict)->'TerraswapAssetDetail':
        # {'Height': '3770968',
        #  'Result': {'assets': [{'amount': '3078494360163',
        #     'info': {'native_token': {'denom': 'uusd'}}},
        #    {'amount': '8189896328',
        #     'info': {'token': {'contract_addr': 'terra137drsu8gce5thf6jr5mxlfghw36rpljt3zj73v'}}}],
        #   'total_share': '155236168311'}}
        asset_detail_1 = TerraswapAssetDetail.from_json(input_json['Result']['assets'][0])
        asset_detail_2 = TerraswapAssetDetail.from_json(input_json['Result']['assets'][1])
        total_share = Decimal(input_json['Result']['total_share'])
        return TerraswapPoolInfo(asset_detail_1=asset_detail_1,
                                 asset_detail_2=asset_detail_2,
                                 total_share=total_share)

In [102]:
pair_contract_to_pool_info_raw = req_all_pools_info.json()['data']

In [103]:
pair_contract_to_pool_info_raw['terra108ukjf6ekezuc52t9keernlqxtmzpj4wf7rx0h']

{'Height': '3904784',
 'Result': '{"assets":[{"amount":"3761580459638","info":{"native_token":{"denom":"uusd"}}},{"amount":"9941707895","info":{"token":{"contract_addr":"terra137drsu8gce5thf6jr5mxlfghw36rpljt3zj73v"}}}],"total_share":"188962888187"}'}

In [104]:
# conver text to json
for c_code,item_dict in pair_contract_to_pool_info_raw.items():
    if isinstance(item_dict.get("Result"),str):
        item_dict['Result'] = json.loads(item_dict['Result'])

In [105]:
pair_contract_to_pool_info : Dict[str,TerraswapPoolInfo] = {c:TerraswapPoolInfo.from_json(item) for c,item in pair_contract_to_pool_info_raw.items()}

In [106]:
pair_contract_to_pool_info['terra108ukjf6ekezuc52t9keernlqxtmzpj4wf7rx0h']

TerraswapPoolInfo(asset_detail_1=TerraswapAssetDetail(token_name_or_contract_address='uusd', is_native_token=True, amount=Decimal('3761580459638')), asset_detail_2=TerraswapAssetDetail(token_name_or_contract_address='terra137drsu8gce5thf6jr5mxlfghw36rpljt3zj73v', is_native_token=False, amount=Decimal('9941707895')), total_share=Decimal('188962888187'))

In [107]:
pair_contract_to_pool_info_raw['terra108ukjf6ekezuc52t9keernlqxtmzpj4wf7rx0h']

{'Height': '3904784',
 'Result': {'assets': [{'amount': '3761580459638',
    'info': {'native_token': {'denom': 'uusd'}}},
   {'amount': '9941707895',
    'info': {'token': {'contract_addr': 'terra137drsu8gce5thf6jr5mxlfghw36rpljt3zj73v'}}}],
  'total_share': '188962888187'}}

In [108]:
pair_contract_to_list_of_tokens :Dict[str,List[str]] = {}
for contract,pool_info in pair_contract_to_pool_info.items():
    first_coin = pool_info.asset_detail_1.token_name_or_contract_address
    second_coin = pool_info.asset_detail_2.token_name_or_contract_address

    pair_contract_to_list_of_tokens[contract] = [first_coin,second_coin]

In [109]:
pair_contract_to_list_of_tokens[list(pair_contract_to_pool_info.keys())[0]]

['uusd', 'terra137drsu8gce5thf6jr5mxlfghw36rpljt3zj73v']

In [110]:
[(c,l) for c,l in pair_contract_to_list_of_tokens.items() if 'uluna' in l and 'uusd' in l]

[('terra1tndcaqxkpc5ce9qee5ggqf430mr2z3pefe5wj6', ['uusd', 'uluna'])]

# Compute Swaps Using Smart Contract Pools

In [117]:
# computations based on the contract terraswap pair contract: 
# https://github.com/terraswap/terraswap/blob/fb33e61672b8cdb036374d96a6e42cfee3c89381/contracts/terraswap_pair/src/contract.rs#L525
def calculate_swap_output(
    input_amount:Decimal,
    input_native_token_name:str=None,
    input_token_contract:str=None,
    output_native_token_name:str=None,
    output_token_contract:str=None,
    commisson_rate:Decimal = Decimal("0.003")
    ):
    # computations based on try_swap function of terraswap pair contract
    input_coin = input_native_token_name or input_token_contract
    output_coin = output_native_token_name or output_token_contract
    assert input_coin!=output_coin
    print("input & output coin ",input_coin,output_coin)
    supporting_pair_contracts = [c for c,token_list in pair_contract_to_list_of_tokens.items() if  input_coin in token_list and output_coin in token_list 
                                   and pair_contract_to_pool_info[c].total_share not in [0,'0',None]
                                ]
    print("found contracts", supporting_pair_contracts)
    if len(supporting_pair_contracts)==1:
        pair_contract = supporting_pair_contracts[0]
        pool_info = pair_contract_to_pool_info[pair_contract]
        print("pool_info",pool_info)
        assert input_coin in [pool_info.asset_detail_1.token_name_or_contract_address,pool_info.asset_detail_2.token_name_or_contract_address]
        assert output_coin in [pool_info.asset_detail_1.token_name_or_contract_address,pool_info.asset_detail_2.token_name_or_contract_address]
        
        input_asset_detail:TerraswapAssetDetail = pool_info.asset_detail_1 if input_coin == pool_info.asset_detail_1.token_name_or_contract_address else pool_info.asset_detail_2
        output_asset_detail:TerraswapAssetDetail = pool_info.asset_detail_1 if output_coin == pool_info.asset_detail_1.token_name_or_contract_address else pool_info.asset_detail_2
        
        input_depth = input_asset_detail.amount
        output_depth = output_asset_detail.amount
        
        
        input_token_decimals = 6 if input_asset_detail.is_native_token is True else token_contract_to_decimals[input_asset_detail.token_name_or_contract_address]
        output_token_decimals = 6 if output_asset_detail.is_native_token is True else token_contract_to_decimals[output_asset_detail.token_name_or_contract_address]
        
        print("input_depth,output_depth",input_depth,output_depth)
        input_amount_considering_decimals = input_amount*Decimal(f'1e{input_token_decimals}')
        
        # // offer => ask
        # // ask_amount = (ask_pool - cp / (offer_pool + offer_amount)) * (1 - commission_rate)
        # let cp = Uint128(offer_pool.u128() * ask_pool.u128());        
        # let return_amount = (ask_pool - cp.multiply_ratio(1u128, offer_pool + offer_amount))?;
        cp = input_depth*output_depth
        output_amount_before_commission = output_depth - cp/(input_depth+input_amount_considering_decimals)
        print("output_amount_before_commission",output_amount_before_commission)
        
        # let commission_amount: Uint128 = return_amount * Decimal::from_str(&COMMISSION_RATE).unwrap();
        commission = commisson_rate*output_amount_before_commission
        print('commission',commission,output_coin)
        # // commission will be absorbed to pool
        # let return_amount: Uint128 = (return_amount - commission_amount).unwrap();
        output_amount_after_commission = output_amount_before_commission - commission
        
        # reduce tax. computation based on compute_tax in terraswap pair contract
        tax_amount = Decimal('0')
        # non-native tokens and luna do not have a tax
        if output_asset_detail.is_native_token is False or (output_asset_detail.is_native_token is True and output_coin =='uluna'):
            tax_amount = Decimal('0')
        else:
            # let tax_rate: Decimal = (terra_querier.query_tax_rate()?).rate;
            # let tax_cap: Uint128 = (terra_querier.query_tax_cap(denom.to_string())?).cap;
            # Ok(std::cmp::min(
            #     (amount
            #         - amount.multiply_ratio(
            #             DECIMAL_FRACTION,
            #             DECIMAL_FRACTION * tax_rate + DECIMAL_FRACTION,
            #         ))?,
            #     tax_cap,
            # ))
            DECIMAL_FRACTION = Decimal('1_000_000_000_000_000_000')
            output_tax_cap = token_to_tax_cap[output_coin] 
            tax_amount_before_cap = output_amount_after_commission - output_amount_after_commission*(DECIMAL_FRACTION/(DECIMAL_FRACTION*tax_rate+DECIMAL_FRACTION))
            tax_amount = min(tax_amount_before_cap,output_tax_cap)
        print("tax_amount",tax_amount)
        print("output_before_tax",output_amount_after_commission)
    
        output_amount_after_commission = output_amount_after_commission - tax_amount
        
        return output_amount_after_commission/Decimal(f'1e{output_token_decimals}')
        
        
    else:
        raise Exception("found two or more")
    # Find the pair contract
    pass

In [118]:
calculate_swap_output(1,input_native_token_name='uluna',output_native_token_name='uusd')

input & output coin  uluna uusd
found contracts ['terra1tndcaqxkpc5ce9qee5ggqf430mr2z3pefe5wj6']
pool_info TerraswapPoolInfo(asset_detail_1=TerraswapAssetDetail(token_name_or_contract_address='uusd', is_native_token=True, amount=Decimal('7668960962068')), asset_detail_2=TerraswapAssetDetail(token_name_or_contract_address='uluna', is_native_token=True, amount=Decimal('892297432032')), total_share=Decimal('1591761811475'))
input_depth,output_depth 892297432032 7668960962068
output_amount_before_commission 8594614.410118085177669
commission 25783.843230354255533007 uusd
tax_amount 40823.169481491424465771239
output_before_tax 8568830.566887730922135993


Decimal('8.528007397406239497670221761')

In [119]:
calculate_swap_output(100,input_native_token_name='uusd',output_native_token_name='uluna')

input & output coin  uusd uluna
found contracts ['terra1tndcaqxkpc5ce9qee5ggqf430mr2z3pefe5wj6']
pool_info TerraswapPoolInfo(asset_detail_1=TerraswapAssetDetail(token_name_or_contract_address='uusd', is_native_token=True, amount=Decimal('7668960962068')), asset_detail_2=TerraswapAssetDetail(token_name_or_contract_address='uluna', is_native_token=True, amount=Decimal('892297432032')), total_share=Decimal('1591761811475'))
input_depth,output_depth 7668960962068 892297432032
output_amount_before_commission 11635028.5445036755803759
commission 34905.0856335110267411277 uluna
tax_amount 0
output_before_tax 11600123.4588701645536347723


Decimal('11.6001234588701645536347723')